In [34]:
import cv2
import pandas as pd
import numpy as np
import keras
import re
import nltk
import string
import json
import matplotlib.pyplot as plt
from time import time
import pickle
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Model, load_model
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Dense, Dropout, Embedding, LSTM
from keras.layers.merge import add

In [35]:

with open('C:\\Users\\abhay\\Desktop\\QNA\\dataq.json','r') as f:
    data = json.load(f)
# with open('data.json','r') as f:
#     data = json.load(f)

In [36]:
d = {}
for i in range(len(data)):
    d[i] = dict(data[i])

In [37]:
data_dict = {}
for key,value in d.items():
    data_dict[d[key]['title']] = d[key]['rules']

In [38]:
def clean_text(sentence):
    sentence = sentence.lower()
    sentence = re.sub("[^a-z]+"," ",sentence)  # taking only alphabets
    sentence = sentence.split()
    sentence = [i for i in sentence if len(i)>1]
    sentence = " ".join(sentence)
    return sentence

In [39]:
for key,caption_list in data_dict.items():
    for i in range(len(caption_list)):
        caption_list[i] = clean_text(caption_list[i])

In [40]:
vocab = set()
for key in data_dict.keys():
    [vocab.update(sentence.split()) for sentence in data_dict[key]]

In [41]:
total_words = []
for key in data_dict.keys():
    [total_words.append(i) for d in data_dict[key] for i in d.split()]

In [42]:
import collections
count = collections.Counter(total_words)
freq_count = dict(count)
# print(freq_count)

In [43]:
sorted_freq = sorted(freq_count.items(),reverse=True,key=lambda x:x[1])
total_words = [i[0] for i in sorted_freq]

In [44]:
word_to_index = {}
for i,word in enumerate(total_words):
    word_to_index[word] = i+1

In [45]:
vocab_size = len(word_to_index) + 1

In [46]:
title_map = {}
title_map_idx = {}
for i,key in enumerate(data_dict.keys()):
    title_map[key] = i+1
    title_map_idx[i] = key

In [47]:
title_map_len = len(title_map)+1

In [48]:
X = []
Y = []
for key,value in data_dict.items():
    for question in data_dict[key]:
        seq = [ word_to_index[word] for word in question.split() if word in word_to_index]
        in_seq = pad_sequences([seq], maxlen=100, value=0, padding='post')[0]
        X.append(in_seq)
        out_seq = to_categorical([title_map[key]], num_classes=title_map_len)[0]
        Y.append(out_seq)
        

In [49]:
X = np.array(X)
Y = np.array(Y)

In [50]:
Y.shape

(911, 135)

In [51]:
input_ques = Input(shape=(100,))
embed = Embedding(input_dim = vocab_size,output_dim=64,mask_zero = True)(input_ques)
lstm1 = LSTM(128,return_sequences = True)(embed)
lstm2 = LSTM(128,return_sequences = False)(lstm1)
dense1 = Dense(128,activation='relu')(lstm2)
output = Dense(title_map_len,activation='softmax')(dense1)

In [52]:
model = Model(inputs=input_ques,outputs = output)

In [53]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 100, 64)           108288    
                                                                 
 lstm_2 (LSTM)               (None, 100, 128)          98816     
                                                                 
 lstm_3 (LSTM)               (None, 128)               131584    
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 135)               17415     
                                                                 
Total params: 372,615
Trainable params: 372,615
Non-trainab

In [54]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [55]:
history = model.fit(X,Y,epochs=50,verbose=1)

Epoch 1/50
29/29 [==============================] - 37s 206ms/step - loss: 4.8546 - accuracy: 0.0252
Epoch 2/50
29/29 [==============================] - 4s 155ms/step - loss: 4.6266 - accuracy: 0.0351
Epoch 3/50
29/29 [==============================] - 4s 139ms/step - loss: 4.2901 - accuracy: 0.0538
Epoch 4/50
29/29 [==============================] - 4s 140ms/step - loss: 3.9253 - accuracy: 0.0790
Epoch 5/50
29/29 [==============================] - 4s 139ms/step - loss: 3.5756 - accuracy: 0.1153
Epoch 6/50
29/29 [==============================] - 4s 136ms/step - loss: 3.1899 - accuracy: 0.1778
Epoch 7/50
29/29 [==============================] - 4s 135ms/step - loss: 2.9159 - accuracy: 0.2228
Epoch 8/50
29/29 [==============================] - 4s 146ms/step - loss: 2.6280 - accuracy: 0.2909
Epoch 9/50
29/29 [==============================] - 4s 135ms/step - loss: 2.3750 - accuracy: 0.3282
Epoch 10/50
29/29 [==============================] - 4s 134ms/step - loss: 2.2118 - accuracy: 0.368

In [56]:
def predict(inp):
    x = []
    inp = clean_text(inp)
    seq = [ word_to_index[word] for word in inp.split() if word in word_to_index]
    print(seq)
    in_seq = pad_sequences([seq], maxlen=100, value=0, padding='post')[0]
    print(in_seq)
    x.append(in_seq)
    x = np.array(x)
    ypred = model.predict(x)
    ypred = ypred.argmax()
    return title_map_idx[ypred-1]
    

In [57]:
print(predict( "What are the points to be considerd for adoption in christian faith?"))
print(predict( "Anything about sex and rape?"))

[3, 14, 1, 919, 4, 8, 920, 6, 181, 7, 107, 451]
[  3  14   1 919   4   8 920   6 181   7 107 451   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]
ADOPTION UNDER CHRISTIAN LAW
[131, 189, 9, 68]
[131 189   9  68   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]
DOWRY DEATH


In [58]:
model.save('checkpoint.h5')

In [59]:
model = load_model('checkpoint.h5')

In [60]:
print(X[0])

[282  31   1  54  43   6  12 115 283 570   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [61]:
predict("How are you?")

[17, 14]
[17 14  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0]


'MAHILA COURTS'

In [62]:
predict("anything else")

[]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


'ACID ATTACKS'

In [63]:
import tensorflow as tf
# These imports are required to load operators' definition.
import tensorflow_text as tf_text
import sentencepiece as spm

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS
]
model_data = converter.convert()
open('model2.tflite','wb').write(model_data)

INFO:tensorflow:Assets written to: C:\Users\abhay\AppData\Local\Temp\tmpul99azf1\assets


INFO:tensorflow:Assets written to: C:\Users\abhay\AppData\Local\Temp\tmpul99azf1\assets


2436604

In [64]:
# import tensorflow as tf
# from tensorflow import lite
# converter =lite.TFLiteConverter.from_keras_model(model)
# converter.experimental_new_converter=True
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
# tf.lite.OpsSet.SELECT_TF_OPS]

# tfmodel = converter.convert()
# open('converted_checkpoint_model.tflite','wb').write(tfmodel)

In [65]:
# from tensorflow.python import keras
# import tensorflow

# modelnew = keras.models.load_model('checkpoint.h5')
# converter=tensorflow.lite.TFLiteConverter.from_keras_model(modelnew)
# tflite_model = converter.convert()
# open("converted_model.tflite", "wb").write(tflite_model)